<a href="https://colab.research.google.com/github/aquapathos/BasicAI/blob/master/AIChallenge001_%E7%94%BB%E5%83%8F%E3%81%AE%E5%8F%8E%E9%9B%86.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 画像の収集
画像を認識するためには、まず事前にたくさんの事例をAIに機械学習させておく必要がありますが、
大量の画像を自分で用意するのは大変なので、プログラムで収集することにします。

以下、重要な指示は<font color='blue'>青字</font>にしてありますので、見落とさないように演習を進めていってください。

# 1. Google Drive をマウント
Google Colaboratory は一定時間経過すると作業内容が削除されてしまいます。消したくないデータや後日再利用したいデータは Google Drive に保存するようにプログラムすることで、残すことができます。


1. <font color='blue'>次のセルを実行この説明の下のセルを実行する
2. リンクと入力フィールドが表示されるのでリンクをクリック
3. 本人認証が要求されるので、応じる。
4. アクセスリクエストを許可するとアクセス用のコードが表示されるのでコピー
5. 下の　Enter your authorization code: の下の枠に貼り付けて Enter
6. 左のファイル一覧エリアで「更新アイコン」（回転マーク）をクリック（エリアが表示されていないなら「ファイルアイコン」（フォルダ型）をクリック）</font>


<img width="390" alt="googlecolab" src="https://user-images.githubusercontent.com/5820803/94802343-739cff00-0422-11eb-8c0d-affa919f8e58.png">




In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# 2. 画像の収集と選別
## 2.1 画像の収集

画像を収集するために必要なライブラリを準備します。  <font color='blue'>次のセルを実行</font>


In [ ]:
import glob
import cv2
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
!pip -q install icrawler
from icrawler.builtin import BingImageCrawler

SIZE = 128 # リサイズ後の画像の高さと幅

機械学習の際に画像のサイズをすべて同じに揃えます。 SIZE はそのサイズです。このサイズを大きくすると画像は鮮明になりますが、学習に時間がかかってしまいます。どれくらいのサイズが適当なのかは、何をしたいかによりますが、対象が何かを見分けるだけならこの程度で十分です。

### 画像収集の例　　ネコの画像をあつめてみる
ネット上で公開されている画像を収集するためのライブラリ、icrawler を使って画像を収集します。
今回は Bing から画像を拾ってくることにします。

次のセルは、猫の画像を最大３００枚まで収集して、一旦 Google Drive の　マイドライブ内の 'tmp/ネコ'　という名のフォルダに保存するプログラムです。　　（ "drive/MyDrive" が Google Driveのマイドライブを表しています）

**keyword= ["ネコ","猫","cat"]** で検索ワードをしてしています。


<font color='blue'>次のセルを実行</font> 　　（３分ぐらいかかります）



In [ ]:
%%time
max_num = 300 # 収集枚数
TMPFOLDER = 'drive/MyDrive/tmp/' # Google ドライブの マイドライブ内の tmp フォルダ
def collectPictures(folderName = "", keywords = [], max_num = max_num):
    crawler = BingImageCrawler(storage={"root_dir":TMPFOLDER+folderName})
    crawler.crawl(keyword= keywords, max_num=max_num)

collectPictures(folderName = "ネコ", keywords = ["ネコ","猫","cat"])

# 2.2 画像の選別

![Google Drive](https://raw.githubusercontent.com/aquapathos/pictures/gh-pages/BasicAIFig001.png)　　　　　　 ![Google Drive](https://raw.githubusercontent.com/aquapathos/pictures/gh-pages/BasicAIFig002.png)   

![Google Drive](https://github.com/aquapathos/pictures/blob/gh-pages/BasicAIFig003.png?raw=true)

1. <font color='blue'>ブラウザで新しいタブをつくり、[Google Drive](https://drive.google.com/) を開きます。
2. **マイドライブ**を開き、**tmp/ネコ** を開きます。
3. リスト名表示からギャラリー表示に切り替えて不適だと思われる画像を削除してください。</font>

不適なのは、次のような画像です。
- ネコが写っていない画像。写っていても小さすぎる画像
- 主要被写体がネコではない画像、つまりネコ以外のものが目立っている画像
- 文字が目立つ画像。できれば文字は入っていないことが望ましいですが、目立たなければOK
- デフォルメされたぬいぐるみやイラスト。
- GIF 動画
- 同一の写真

パット見て、これは「ネコ」が主題の写真だと感じられるものを残して、そうでないもの、他のモノが多くの面積を締めているものは削除してください。(厳密なルールはありません。あなた自身がこれは何かと問われて「ネコ」と答える写真を選べばいいです。あなたの選んだものが教師データとなります。)

# 2.3　画像のリサイズ

何度も使うので関数を定義しておくことにする．  <font color='blue'>次のセルを実行</font>

In [ ]:
def resize(foldername, size=SIZE):
    imgnames = glob.glob(foldername+"/*") # 画像ファイル名のリスト
    images = []
    for imgname in imgnames:
        img = cv2.cvtColor(np.array(Image.open(imgname).convert('RGBA')),cv2.COLOR_RGBA2BGR)
        height = img.shape[0]
        width = img.shape[1]
        if height > width :
            m = (height - width)//2 
            img = img[m:m+width]
        else:
            m = (width - height)//2
            img = img[:,m:m+height]
        img = cv2.resize(img, (SIZE,SIZE))
        images.append(img)
    return images

さっそくリサイズしてみましょう。　　<font color='blue'>次のセルを実行</font>

In [ ]:
# リサイズを実行
c1img = resize(TMPFOLDER+"ネコ")
print("ネコ - ", len(c1img),"枚の画像がリサイズされました")

上のプログラムでは、ネコの画像をリサイズし、その集まりを、c1img と名付けています。

表示用関数を定義し、表示して確認してみます。　 <font color='blue'>次のセルを実行</font>

In [ ]:
import math
# start番からnpic枚表示する関数を定義
plt.rcParams['figure.figsize'] = (12.0, 7.0)
def showimg(images, start = 0, npic = 48):
    n = npic if len(images) >= start+npic else len(images) - start
    plt.figure(figsize=(8,7.5*(math.ceil(n/8))/6),dpi=150)
    i = 0
    while True:
        if i < n :  
            plt.subplot((n-1)//8+1,8,i+1)
            plt.xticks([])
            plt.yticks([])
            plt.imshow(images[start+i][:,:,::-1])
            plt.title("{}".format(start+i))
            i += 1
        else:
            break

さっそく表示させてみましょう。　１００枚まで表示と指定していますが、たくさん削除していたら１００枚ないかもしれません。　<font color='blue'>次のセルを実行</font>

In [ ]:
showimg(images = c1img,start=0,npic=100) # c1img の 最初（０番）から100枚までを表示

## 2.4　　不適切画像を再チェック
Google Drive 上で削除漏れした画像や同一の写真が複数枚あることに気づいた場合のために、番号を指定して削除する関数を定義しておきましょう。

　　<font color='blue'>次のセルを実行</font>

In [ ]:
from copy import copy

# メモリ上で番号指定で画像を削除する関数
def delimages(images, dellist=[]):
    newimg = copy(images)
    if len(dellist)>0:
        for i, img in enumerate(dellist):
            del newimg[img-i]
    if len(dellist)>0 and len(newimg) > dellist[0]:
        showimg(newimg,dellist[0])
    return newimg

# 削除関数の使用例
0 、１、２０、２５番を削除したい場合は次のコマンドを実行します。  
**delimages(c1img, [0,1,20,25])**

<font color='red'>注意 削除する番号は必ず昇順とすること．[20,25,1,0] のように大きい番号のあとに小さい番号が来てはいけない．</font>


削除したい画像がある場合は、次のセルの [] にその番号をカンマ(,)で区切って並べ、実行してください。

In [ ]:
c1img = delimages(c1img, [])

# 2.5　　同様にして別のカテゴリの画像を集める．

こんどはイヌの画像を集めましょう。 　<font color='blue'>次のセルを実行</font>

In [ ]:
%%time
collectPictures(folderName = "イヌ", keywords = ["イヌ","犬","dog"])

## ネコと同じ要領で画像を選別

<font color='blue'>google drive 上で画像を選別し、不適切な画像を削除してください。</font>　<font color='blue'>削除後、次のセルを実行</font>

In [ ]:
c2img = resize(TMPFOLDER+"イヌ")
print("イヌ - ", len(c2img),"枚の画像がリサイズされました")

In [ ]:
showimg(images = c2img,start = 0,npic = 100)

削除したい画像がある場合は、次のセルの [] にその番号をカンマ(,)で区切って並べ、実行してください。



In [ ]:
c2img = delimages(c2img, [])

# ３．　学習用画像データの保存と読み込み
サイズが正規化され、選別ずみの画像はこの時点で c1img, c2img という名の python オブジェクト内に入っており、メモリ上に置かれています。

これらはGoogle Colaboratory を終了したり、一定時間経過すると消えてしまい、再利用できません。　

## 3.1 データの保存と読み出しの関数を定義
python にはメモリ上に置かれた変数の値をそっくりそのままファイルとして保存する関数が用意されていますので、その関数を使って変数の中身を Google ドライブに保存したり、読み出したりできるようにしておきます。　　　<font color='blue'>次のセルを実行</font>

In [ ]:
import pickle
import os

# 画像データを pickle 形式で保存
def storeCategoryImages(cat, fname, folder = "."):
    os.makedirs(folder, exist_ok=True)
    f = open(folder+"/"+fname,'wb')
    pickle.dump(cat,f)
    f.close
    
# pickle 形式で保存された画像データの読み込み
def loadCategoryImages(fname, folder = "."):
    f = open(folder+"/"+fname,'rb')
    cat = pickle.load(f)
    f.close
    return cat

## 3.2 変数内のデータをファイルとして出力

さっそく c1img（ネコ画像の集まり） と c2img（イヌ画像の集まり）を　pickle 形式で保存しましょう。　　　<font color='blue'>次のセルを実行</font>

In [ ]:
GFOLDER = "drive/MyDrive/LDATA"  # データ保存用のフォルダ名
storeCategoryImages(c1img,"ネコ.pkl",folder=GFOLDER)
storeCategoryImages(c2img,"イヌ.pkl",folder=GFOLDER)

# 3.2 学習用画像データの読み込み

今度は逆に、pickle 形式で保存されたデータを変数に読み込み、正しく復元できるか確認しておきましょう。　　<font color='blue'>次のセルを実行</font>

In [ ]:
testimg = loadCategoryImages("ネコ.pkl", folder=GFOLDER)
showimg(images = testimg,start = 3)

# <font color='red'>課題

ネコやイヌの例をまねて、「カメ」のデータを収集し、カメ.pkl を作りなさい。</font>

# 「[AIChallenge002-学習と識別](https://github.com/aquapathos/BasicAI/blob/master/5CNN/AIChallenge002_%E5%AD%A6%E7%BF%92%E3%81%A8%E8%AD%98%E5%88%A5.ipynb)」へつづく

# 課題実行のためのヒント
画像の収集

In [ ]:
# collectPictures(folderName = "カメ", keywords = ["カメ","亀","turtle"])
# collectPictures(folderName = "ニワトリ", keywords = ["ニワトリ","鶏","chicken"])
# collectPictures(folderName = "ハムスター", keywords = ["ハムスター","hamster"])

リサイズ

In [ ]:
# c3img = resize(TMPFOLDER+"カメ")
# print("亀 - ",len(c3img),"枚の画像がリサイズされました")
# c4img = resize(TMPFOLDER+"ニワトリ")
# print("ニワトリ - ",len(c4img),"枚の画像がリサイズされました")
# c5img = resize(TMPFOLDER+"ハムスター")
# print("ハムスター - ",len(c5img),"枚の画像がリサイズされました")

データの保存

In [ ]:
# storeCategoryImages(c3img,"カメ.pkl",folder=GFOLDER)
# storeCategoryImages(c4img,"ニワトリ.pkl",folder=GFOLDER)
# storeCategoryImages(c5img,"ハムスター.pkl",folder=GFOLDER)